In [4]:
import os

In [11]:
base_config = {
    "saudio_option": {
        "input_port": "UGREEN CM564 USB Audio  Mono",
        "output_port": "USB2.0 Device Analog Stereo",
        "ringbuffer_size": 1280
    },
    "ofdm_option": {
        "symbol_freq": 1000,
        "symbol_bits": 1,
        "channels": [
            3,
            7
        ],
        "shift_samples": 0
    },
    "sphy_option": {
        "bin_payload_size": 40,
        "frame_gap_size": 48
    },
    "project1_option": {
        "payload_size": 1
    }
}

In [6]:
import copy
import json
import itertools

config_result_mp = {}

In [12]:
freqs = [1000, 2000, 3000, 4000]
max_freq = 9000


for freq in freqs:
    channels = [i for i in range(1, 13) if i * freq <= max_freq]
    channel_tuples = itertools.combinations(channels, 2)

    for channel in channel_tuples:
        import copy
        config = copy.deepcopy(base_config)
        config["ofdm_option"]["symbol_freq"] = freq
        config["ofdm_option"]["channels"] = list(channel)
        import json
        # write to /home/ryo/code/supersonic/build/config.json
        json.dump(config, open("/home/ryo/code/supersonic/build/config.json", "w"), indent=4)

        print(f"freq: {freq}, channel: {channel}")

        os.system("cd /home/ryo/code/supersonic/build && rm input.txt")
        os.system("cd /home/ryo/code/supersonic/build && rm output.txt")
        os.system("cd /home/ryo/code/supersonic/build && ./supersonic/sendrecv -t 3 > /dev/null 2> /dev/null")
        os.system("cd /home/ryo/code/supersonic/build && python ../jupyter/mdiff.py > result.txt")

        with open("/home/ryo/code/supersonic/build/result.txt", "r") as f:
            result = f.read()
            config_result_mp[(freq, channel)] = result
            print(f"freq: {freq}, channel: {channel}, result: {result}")

freq: 1000, channel: (1, 2)
freq: 1000, channel: (1, 2), result: 0.978

freq: 1000, channel: (1, 3)
freq: 1000, channel: (1, 3), result: 0.976

freq: 1000, channel: (1, 4)
freq: 1000, channel: (1, 4), result: 1.0

freq: 1000, channel: (1, 5)
freq: 1000, channel: (1, 5), result: 0.984

freq: 1000, channel: (1, 6)
freq: 1000, channel: (1, 6), result: 0.974

freq: 1000, channel: (1, 7)
freq: 1000, channel: (1, 7), result: 1.0

freq: 1000, channel: (1, 8)
freq: 1000, channel: (1, 8), result: 0.974

freq: 1000, channel: (1, 9)
freq: 1000, channel: (1, 9), result: 0.976

freq: 1000, channel: (2, 3)
freq: 1000, channel: (2, 3), result: 1.0

freq: 1000, channel: (2, 4)
freq: 1000, channel: (2, 4), result: 0.974

freq: 1000, channel: (2, 5)
freq: 1000, channel: (2, 5), result: 0.976

freq: 1000, channel: (2, 6)
freq: 1000, channel: (2, 6), result: 1.0

freq: 1000, channel: (2, 7)
freq: 1000, channel: (2, 7), result: 1.0

freq: 1000, channel: (2, 8)
freq: 1000, channel: (2, 8), result: 0.974

fr

In [13]:
def parse_output(output):
    output = output.strip()
    lines = output.split("\n")
    correct_rate = float(lines[0])
    has_error = 'error' in output
    return correct_rate, has_error

config_result_list = [(k, v) for k, v in config_result_mp.items()]

sorted_config_result_list = sorted(config_result_list, key=lambda x: parse_output(x[1])[0], reverse=True)

for k, v in sorted_config_result_list:
    correct_rate, has_error = parse_output(v)
    print(k, correct_rate, has_error)

(1000, (1, 4)) 1.0 False
(1000, (1, 7)) 1.0 False
(1000, (2, 3)) 1.0 False
(1000, (2, 6)) 1.0 False
(1000, (2, 7)) 1.0 False
(1000, (2, 9)) 1.0 False
(1000, (3, 4)) 1.0 False
(1000, (3, 7)) 1.0 False
(1000, (3, 9)) 1.0 False
(1000, (4, 5)) 1.0 False
(1000, (4, 7)) 1.0 False
(1000, (4, 9)) 1.0 False
(1000, (5, 7)) 1.0 False
(1000, (6, 8)) 1.0 False
(1000, (7, 8)) 1.0 False
(2000, (1, 2)) 1.0 False
(2000, (1, 3)) 1.0 False
(2000, (1, 4)) 1.0 False
(2000, (2, 3)) 1.0 False
(3000, (1, 2)) 1.0 False
(4000, (1, 2)) 1.0 False
(2000, (3, 4)) 0.998 False
(1000, (5, 9)) 0.992 False
(3000, (1, 3)) 0.986 False
(1000, (1, 5)) 0.984 False
(3000, (2, 3)) 0.982 False
(1000, (1, 2)) 0.978 False
(1000, (1, 3)) 0.976 False
(1000, (1, 9)) 0.976 False
(1000, (2, 5)) 0.976 False
(1000, (3, 5)) 0.976 False
(1000, (3, 8)) 0.976 False
(1000, (4, 6)) 0.976 False
(1000, (6, 7)) 0.976 False
(1000, (1, 6)) 0.974 False
(1000, (1, 8)) 0.974 False
(1000, (2, 4)) 0.974 False
(1000, (2, 8)) 0.974 False
(1000, (4, 8)) 0